In [1]:
import numpy as np
import pandas as pd
import nltk
import random
import string
import re, string, unicodedata
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import wikipedia as wk
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")
nltk.download('punkt') 
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
nltk.download('averaged_perceptron_tagger')
import csv
import itertools

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
import firebase_admin
import google.cloud
from firebase_admin import credentials, firestore
from firebase_admin import db

if (not len(firebase_admin._apps)):
    cred = credentials.Certificate(r"C:\Users\ADMIN\Documents\Alorine\alorinedotcom-firebase-adminsdk-ii0g7-d2bc0817d1.json")
    default_app = firebase_admin.initialize_app(cred)

db = firestore.client()
posts_table = db.collection(u'posts')
res = rec = []
google = raw = ''
try:
    posts_data = posts_table.stream()
    for each_post in posts_data:
        pst = each_post.to_dict()
        list = [[k, v] for k, v in pst.items()]
        output = [item for item in list if (item[0] == 'title' or item[0] == 'category' or item[0] == 'authorUid' or item[0] == 'hits')]
        res = res + output
except google.cloud.exceptions.NotFound:
    print(u'Missing data')   

def extract_key(v):
    return v[0]
res = sorted(res, key=extract_key)
result = [[x[1] for x in g] for k, g in itertools.groupby(res, extract_key)]


In [3]:
p = q = r = s = t = u = []
i = 0
m = ''
pattern = re.compile(r'(,\s){2,}')
try:
    posts_data = posts_table.stream()
    for each_post in posts_data:
        pst = each_post.to_dict()
        list = [[k, v] for k, v in pst.items()]
        p = [item for item in list if (item[0] == 'bodyJson')]
        q = q + p
except google.cloud.exceptions.NotFound:
    print(u'Missing data')
r = [[x[1] for x in g] for k, g in itertools.groupby(q, extract_key)]    

for lists in r:
    lists = [i.strip("[]").split(", ") for i in lists] 
    s = s + lists

for lists in s:
    b = ''
    for item in lists:
        item = item.replace('{"insert":', ' ')
        item = item.replace('"attributes":{"block":"ul"}}', ' ')
        item = item.replace('"attributes":{"block":"ol"}}', ' ')
        item = item.replace('\\n',' ')
        item = re.sub("[^!,a-z'A-Z.]", " ", item)
        item = re.sub(pattern, ' ', item)
        item = re.sub(',+', ' ',item)
        item = item.replace('attributes','')
        item = re.sub(' +', ' ',item)
        b = b + " " + item
    b = b.lower()
    t.insert(i, b)
    m += b
    i += 1

result.append(t)


In [4]:
from itertools import zip_longest
export_data = zip_longest(*result, fillvalue = '')
csvfile=open('data.csv', 'w', newline='', encoding="utf-8")
with open('ALORINE_data.csv', 'w', encoding="utf-8", newline='') as myfile:
      wr = csv.writer(myfile, delimiter=',')
      wr.writerow(("title", "category", "authorUid", "hits", "bodyJson"))
      wr.writerows(export_data)
        
myfile.close() 

In [5]:
raw = m.lower()
sent_tokens = nltk.sent_tokenize(raw)

def Normalize(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    #word tokenization
    word_token = nltk.word_tokenize(text.lower().translate(remove_punct_dict))
    
    #remove ascii
    new_words = []
    for word in word_token:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    
    #Remove tags
    rmv = []
    for w in new_words:
        text=re.sub("&lt;/?.*?&gt;","&lt;&gt;",w)
        rmv.append(text)
        
    #pos tagging and lemmatization
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    lmtzr = WordNetLemmatizer()
    lemma_list = []
    rmv = [i for i in rmv if i]
    for token, tag in nltk.pos_tag(rmv):
        lemma = lmtzr.lemmatize(token, tag_map[tag[0]])
        lemma_list.append(lemma)
    return lemma_list

In [6]:
welcome_input = ("hello", "hi", "greetings", "sup", "what's up","hey",)
welcome_response = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def welcome(user_response):
    for word in user_response.split():
        if word.lower() in welcome_input:
            return random.choice(welcome_response)

In [7]:
def generateResponse(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=Normalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    #vals = cosine_similarity(tfidf[-1], tfidf)
    vals = linear_kernel(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0) or "tell me about" in user_response:
        print("Just a moment...")
        if user_response:
            robo_response = wikipedia_data(user_response)
            return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response
#wikipedia search
def wikipedia_data(input):
    reg_ex = re.search('tell me about (.*)', input)
    try:
        if reg_ex:
            topic = reg_ex.group(1)
            wiki = wk.summary(topic, sentences = 3)
            return wiki
    except Exception as e:
            print("I apologise! I don't understand.")

In [8]:
def recommendation(rec):
    if isinstance(rec, str) == True:
        with open('ALORINE_data.csv', 'r', encoding="utf-8") as file:
            reader = csv.reader(file, delimiter=',')
            for row in reader:
                if rec in row[4]:
                    print("Recommendation: "+row[3])
        
    

In [9]:
flag=True
print("Hello! I am Alina and I'm here to answer your queries. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response not in ['bye','shutdown','exit', 'quit']):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Alina : You are welcome..")
        else:
            if(welcome(user_response)!=None):
                print("Alina : "+welcome(user_response))
            else:
                print("Alina : ",end="")
                rec = generateResponse(user_response)
                print(rec)
                sent_tokens.remove(user_response)
                recommendation(rec)             
    else:
        flag=False
        print("Alina : Bye!!! ")

Hello! I am Alina and I'm here to answer your queries. If you want to exit, type Bye!
Hello
Alina : hi
Tell me about Health
Alina : Just a moment...
Health is a state of physical, mental and social well-being in which disease and infirmity are absent.


== History ==
The meaning of health has evolved over time. In keeping with the biomedical perspective, early definitions of health focused on the theme of the body's ability to function; health was seen as a state of normal function that could be disrupted from time to time by disease.
what are the popular dieting myths?
Alina : here are the most popular diet myths night time eating makes you fat night and day have nothing to do with your weight gain.
Recommendation: 3 Popular dieting myths
Thank you
Alina : You are welcome..
